# Training
This notebook is for reproducing the training that led to the results presented in the thesis. For this, the dataset should be saved as specified in the ``README``. Then, the following three steps are taken to enable the training:
1. The images are split into train, validation and test set and divided into smaller image patches. The JSON files that transform the data into COCO format are generated.
2. By specifying the experiment, for which the training shall be reproduced, the respective pre-trained COCO weights are downloaded and the configuration for the experiment is set.
3. The dataset is built and training starts. The model is evaluated and checkpoints are created after one, respectively three epochs (depending on the experiment).

## 0: Check Dependencies

In [4]:
import sys
import os
import mmcv

ROOT_DIR = os.path.abspath("..")
sys.path.append(ROOT_DIR)

# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__) # should be 2.25.0

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version
print(get_compiling_cuda_version()) # should be matching Pytorch's cuda version

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
1.10.1+cu111 False
2.25.0
11.1


## 1: Create Dataset

If the following values (except for the paths) are left unchanged, image patches and JSON files are created so that all experiments barring the one that uses image patches with a minimum scale of 150 pixels (modification \[b\]) can easily be reproduced.

In [5]:
from DataInitialization import DataInitialization

# Path to the folder containing the original satellite images
INPUT_IMG_DIR = os.path.abspath("./data/Training_Images_RGB")
# Path to the folder containing the `.shp` annotations
ANNO_DIR = os.path.abspath("./data/Training_tree_polygons")
# Folder where train, val and test sets are created
IMG_DIR = os.path.dirname(INPUT_IMG_DIR)

# If sets are not created randomly, these indices mark the images belonging to the validation set
# and test set
val_images = [3, 8, 15, 16, 20, 28, 33, 35, 40, 43, 52, 60, 65, 68, 78, 79, 83, 89, 95, 98, 100, 107]
test_images = [5, 7, 17, 18, 24, 30, 41, 45, 47, 49, 51, 59, 62, 70, 76, 77, 81, 91, 99, 104, 105, 108]

# Width and height of the image patches. If a list is passed, patches of multiple sizes are created for
# the training set. For the validation and test set, only image patches of the first size are created. 
min_width = [300, 600]
min_height = [300, 600]
# Which subsets to create. 
subsets = ['train', 'val', 'test']

# Set up initialization of the datasets. If ``force`` is False, for each subset image patches and JSON
# files are only created if they do not exist thus far. If set to true, additional image patches are
# created and JSON files are regenerated, including all image patches.
ri = DataInitialization(img_dir=INPUT_IMG_DIR,
                          img_output_dir=IMG_DIR,
                          subsets=subsets,
                          force=False)

# Method to create image patches
ri.split_crop(min_width=min_width, min_height=min_height, random_split=False, val_images=val_images, test_images=test_images)
# Method to create JSON files in COCO format
ri.load_rwanda_data(anno_dir=ANNO_DIR)
ri.load_rwanda_data(anno_dir=ANNO_DIR, load_prefix="300", subsets=['train'])

Splitting all input images into tiles of 300 x 300
Splitting train input images into tiles of 600 x 600
0 image patches created


## 2: Load Experiment for Training
To reproduce the training of the different models evaluated in the thesis, their configurations can be loaded by specifying the experiment name. A list of all the experiment_names can be generated with ``print_experiments()``.


If the widths and heights of the image patches or the load prefixes were changed from the default values in the data creation step, the paths to the JSON files in ``cfg`` might need to be updated.

In [1]:
from ModelInitialization import ModelInitialization

ModelInitialization.print_experiments()

MaskR50_-
MaskR101_-
MaskR50_1
MaskR50_2
MaskR50_3
MaskR50_4
MaskR50_5
MaskR50_6
MaskR50_7
CascadeR50_-
CascadeR101_-
MaskR50_a
MaskR50_ab
MaskR50_ac
MaskR50_acd
MaskR50_acde
CascadeR50_acd


In [6]:
from ModelInitialization import ModelInitialization

# Specify which training shall be reproduced.
experiment = "MaskR50_acd"

mi = ModelInitialization(experiment=experiment)
model_dir = "./pretrained_models"

# Downloads the pretrained model files if not already downloaded
checkpoint = mi.load_pretrained_model(model_dir=model_dir)
# Loads configuration for chosen experiment
cfg = mi.load_config(anno_dir=ANNO_DIR, img_dir=IMG_DIR, checkpoint=checkpoint)

MaskR50 already downloaded.
Loaded configuration for MaskR50_acd.


In [ ]:
# Optional: Print Confiuration to check or change specific parameters.
# print(f'Config:\n{cfg.pretty_text}')

## 3: Train Model

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import pickle

# Set seed to reproduce results
set_random_seed(cfg.seed, deterministic=False)

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))

train_detector(model, datasets, cfg, distributed=False, validate=True)

# After Training, save the configuration file
with open(cfg.work_dir + '/cfg.pkl','wb') as f:
    pickle.dump(cfg, f)